# CRYSTALS Dilithium

O CRYSTALS Dilithium trata-se de uma técnica de assinatura digital pós-quântica que recorre a R-LWE. Esta foi proposta ao concurso de definição de standards para a criptografia Pós-Quântica iniciada pelo NIST e passou para a segunda fase. 

## Parâmetros da técnica

Neste caso, foi implementada uma classe que efetua a assinatura e verificação de mensagens recorrendo ao modo recomendado da técnica. Como tal, definem-se os seguintes parâmetros:
+ Um primo `q = 2^23 - 2^13 + 1`.
+ Um inteiro `d = 14`.
+ Um inteiro `γ1 = (q - 1) / 16`.
+ Um inteiro `γ2 = γ1 / 2`.
+ Um par de inteiros `(k,l) = (5,4)`.
+ Um inteiro `η = 5`.
+ Um inteiro `β = 275`.
+ Um inteiro `ω = 96`

In [ ]:
import hashlib
import numpy
import random

q = 8380417
d = 14
gamma1 = 523776
gamma2 = 261888 # gamma1/2
k = 5
l = 4
n = 5
beta = 275
omega = 96

## Anéis de polinómios

Definem-se os anéis `Z[x]` e `(Z/q)[x]/(x^N + 1)` como `Rq`.

In [ ]:
Z.<x>  = ZZ[]        # polinómios de coeficientes inteiros
Rq.<x>  = PolynomialRing(GF(q),name='x').quotient(x^n+1)

def small(r,n):
        return Z([numpy.random.randint(-n, high=n) for _ in range(r)])

## Classe Dilithium

A classe Dilithium contém toda a lógica necessária para a elaboração de pares de chaves, assinatura e verificação de mensagens.

In [ ]:
class Dilithium:
    def __init__(self):
        p = random.getrandbits(256)
        K = getrandbits(256)
        s1 = small(l,n)
        s2 = small(k,n)
            
        self.A = matrix(Rq, k, l, lambda i, j: Rq.random_element())
        t = self.A * s1 + s2
        (t1, t0) = power2round(t,d)
        tr = CRH(p, t1) # tr pertence a {0, 1}^384
        
        self.pk = (p, t1)
        self.sk = (p, K, tr, s1, s2, t0)
        
    def sign(self, m):
        A = ExpandA(p)
        u = CRH(tr, M)
        k = 0
        (z,h) = None
        pp = CRH(K, u) # pp pertence a {0, 1}^384 (também é possível ser um aletório de 384 bits)
        p, K, tr, s1, s2, t0 = self.sk
        
        while((z,h) == None):
            y = expandMask(pp, k) #s1 # ajustado a gamma1 - 1
            w = A * y
            w1 = highBits(w, 2 * gamma2)
            c = H(M, w1) # pertencente a B60
            z = y + c * s1
            (r1, r0) = decompose(w - c * s2, 2 * gamma2)
            if (matrix.norm(z) >= (gamma1 - beta)) or matrix.norm(r0) >= (gamma2 - beta) or r1 != w1:
                (z,h) = None
            else:
                h = makeHint(-c * t0, w - c* s2 + c * t0, 2 * gamma2)
                if matrix.norm(c*t0) >= gamma2: #or número de 1's em h > omega:
                    (z,h) = None
            k = k + 1
        return (z,h,c)
    
    def verify(self, M, signature):
        p, t1 = self.pk
        z, h, c = signature
        A = ExpandA(p)
        u = CRH(CHR(p, t1), M)
        
        w1 = useHint(h, A*z - c*t1 * 2**d, 2*gamma2)
        return matrix.norm(z) < gamma1 - beta and c == H(M, w1) # and numero de 1's em h <= omega
    
    def decompose(r, alpha):
        r = r // q
        r0 = r // alpha
        if((r - r0) == (q - 1)):
            r1 = 0 
            r0 = r0 - 1
        else:
            r1 = (r - r0)/alpha
        return (r1, r0)
            
    def highBits(r,alpha):
        (r1, r0) = decompose(r,alpha)
        return r1
    
    def power2round(r,d):
        r = r // q
        r0 = r // (d**2)
        return ((r - r0) / (d**2), r0)
    
    def makeHint(z,r,alpha):
        r1 = highBits(r,alpha)
        v1 = highBits(r + z, alpha)
        return r1 == v1
    
    def useHint(h,r,alpha):
        m = (q - 1)/alpha
        (r1, r0) = decompose(r,alpha)
        if h == 1 and r0 > 0:
            return (r1 + 1) // m
        if h == 1 and r0 <= 0:
            return (r1 - 1) // m
        return r1
    
    def expandA(self, p):
        A = matrix(Rq, k, l, lambda i, j: aux(i,j,self.A[i,j]))              
        
        shake.update(p*self.A)
        return shake.digest()
        
    def aux(i, j, aij):
        shake = hashlib.shake_128()
        shake.update((256 * i + j) * p * self.A[i,j])
        return shake.digest()
        
        
    def expandMask(p, k):
        shake = haslib.shake_256()
        y = small(l,gamma1 - 1)
        shake.update(p*k*y)
        
        return shake.digest()
        
    def CRH(a, b):
        shake = hashlib.shake_256()
        shake.update(a)
        skake.update(b)
        hash = shake.digest()
        return hash[:48]
    
    def H(a,b):
        shake = hashlib.shake_256()
        shake.update(a)
        shake.update(b)
        return shake.digest()[:60]

In [ ]:
d = Dilithium()